# Step 1 - Clean data

In [27]:
import pandas as pd

## 1.1 Clean data for Property Assessment DB

In [28]:
# Load the data from data/property_assessment.csv
pa_df = pd.read_csv('data/property_assessment.csv')
pa_df.head()

,PID,CM_ID,GIS_ID,ST_NUM,ST_NAME,UNIT_NUM,CITY,ZIPCODE,BLDG_SEQ,NUM_BLDGS,...,KITCHEN_STYLE2,KITCHEN_STYLE3,HEAT_TYPE,HEAT_FUEL,AC_TYPE,FIRE_PLACE,ORIENTATION,NUM_PARKING,PROP_VIEW,CORNER_UNIT
0,100001000,,100001000.0,104 A 104,PUTNAM ST,NaN,EAST BOSTON,2128,1.0,1,...,S - Semi-Modern,S - Semi-Modern,W - Ht Water/Steam,NaN,N - None,0,NaN,3,A - Average,NaN
1,100002000,,100002000.0,197,LEXINGTON ST,NaN,EAST BOSTON,2128,1.0,1,...,M - Modern,M - Modern,F - Forced Hot Air,NaN,C - Central AC,0,NaN,0,A - Average,NaN
2,100003000,,100003000.0,199,LEXINGTON ST,NaN,EAST BOSTON,2128,1.0,1,...,S - Semi-Modern,S - Semi-Modern,S - Space Heat,NaN,N - None,0,NaN,0,A - Average,NaN
3,100004000,,100004000.0,201,LEXINGTON ST,NaN,EAST BOSTON,2128,1.0,1,...,S - Semi-Modern,S - Semi-Modern,W - Ht Water/Steam,NaN,N - None,0,NaN,0,A - Average,NaN
4,100005000,,100005000.0,203,LEXINGTON ST,NaN,EAST BOSTON,2128,1.0,1,...,S - Semi-Modern,NaN,W - Ht Water/Steam,NaN,N - None,0,NaN,0,A - Average,NaN


In [29]:
# Select columns we are interested in
interested_columns = ["OWN_OCC", "LU_DESC", "UNIT_NUM", "ST_NAME"]
property_assessment = pa_df[interested_columns]
property_assessment.head()

,OWN_OCC,LU_DESC,UNIT_NUM,ST_NAME
0,Y,THREE-FAM DWELLING,NaN,PUTNAM ST
1,Y,THREE-FAM DWELLING,NaN,LEXINGTON ST
2,N,THREE-FAM DWELLING,NaN,LEXINGTON ST
3,N,THREE-FAM DWELLING,NaN,LEXINGTON ST
4,Y,TWO-FAM DWELLING,NaN,LEXINGTON ST


## 1.2 Clean data for Income-Restricted Housing DB

In [30]:
# Load the data from data/income_restricted.csv
ir_df = pd.read_csv('data/income_restricted.csv')
ir_df.head()

,Project Name,Neighborhood,Zip Code,TtlProjUnits,RentUnits,OwnUnits,TtlMarket,MarketRent,MarketOwn,Total Income-Restricted,Income-Restricted Rental,Income-Restricted Ownership,Tenure,Public/ Private,Includes Senior Units?,Section 8
0,Abbot Street/ Shawmut Ave,Roxbury,2119.0,16,16,0,0.0,0.0,0.0,16,16,0.0,Rental,Private,NaN,NaN
1,Academy Homes I,Roxbury,2119.0,202,202,0,52.0,52.0,0.0,150,150,0.0,Rental,Private,NaN,NaN
2,Academy Homes II,Roxbury,2119.0,236,236,0,0.0,0.0,0.0,236,236,0.0,Rental,Private,NaN,Y
3,Adams Court Phase A,Mattapan,2126.0,50,50,0,0.0,0.0,0.0,50,50,0.0,Rental,Private,NaN,NaN
4,Adams Court Phase B,Mattapan,2126.0,45,45,0,0.0,0.0,0.0,45,45,0.0,Rental,Private,NaN,NaN


In [31]:
# Select columns we are interested in
interested_columns = ["Project Name"]
income_restricted = ir_df[interested_columns]
income_restricted.head()

,Project Name
0,Abbot Street/ Shawmut Ave
1,Academy Homes I
2,Academy Homes II
3,Adams Court Phase A
4,Adams Court Phase B


In [1]:
%pip install python-dotenv

  Using cached python_dotenv-0.21.0-py3-none-any.whl (18 kB)
You should consider upgrading via the '/usr/local/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [32]:
# Derive street name from Project Name using Google's Place API
import requests
import os

# Load .env file
from dotenv import load_dotenv
load_dotenv()

URL = "https://maps.googleapis.com/maps/api/place/findplacefromtext/json"
API_KEY = os.environ["GOOGLE_API_KEY"]

def derive_street_name(project_name):
    res = requests.get(
        URL,
        params={
            "input": project_name,
            "inputtype": "textquery",
            "fields": "formatted_address",
            "key": API_KEY,
        }
    )

    addresses = res.json()["candidates"]
    if len(addresses) == 0:
        return None

    if "formatted_address" not in addresses[0]:
        return None
    
    return addresses[0]["formatted_address"]
    
income_restricted["ST_NAME"] = income_restricted["Project Name"].apply(derive_street_name)

# Save as csv file
income_restricted.to_csv("data/income_restricted_with_st.csv", index=False)

income_restricted.head()

/var/folders/f1/fbz7n7h1591013lxg3r1w2780000gn/T/ipykernel_3024/844086887.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  income_restricted["ST_NAME"] = income_restricted["Project Name"].apply(derive_street_name)


,Project Name,ST_NAME
0,Abbot Street/ Shawmut Ave,"100 Shawmut Ave, Boston, MA 02118, United States"
1,Academy Homes I,"1592 Columbus Ave, Roxbury, MA 02119, United S..."
2,Academy Homes II,"2926 Washington St, Roxbury, MA 02119, United ..."
3,Adams Court Phase A,"1 Pemberton Square, Boston, MA 02108, United S..."
4,Adams Court Phase B,"1 Pemberton Square, Boston, MA 02108, United S..."
